Як змінюється генерація електроенергії з різних джерел впродовж доби?

In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
data = pd.read_excel("../data/2014-2020.xlsx")[["Час/Дата", "AES", "TEC", "VDE", "TES", "GES", "GAES_GEN"]]
data.head()

Час/Дата   AES   TEC  VDE   TES     GES  GAES_GEN
0  24-31.12.2020  9235  2039  621  4942   385.0         0
1  23-31.12.2020  9221  2159  707  5549   470.0         0
2  22-31.12.2020  9249  2377  709  5906  1000.0         0
3  21-31.12.2020  9256  2499  702  6329   909.0       322
4  20-31.12.2020  9213  2521  702  6640   823.0       602

In [4]:
# перетворюємо стрічку з датою і годиною в datetime
# в даних 12 година ночі позначається як 24, pandas ця годину позначає як 0
data['date'] = pd.to_datetime(data['Час/Дата'].replace(regex=r'^24-', value='0-'), format='%H-%d.%m.%Y')
data = data.drop(columns=["Час/Дата"])

In [5]:
# виділяємо годину в окрему колонку
data['hour'] = data['date'].dt.hour

In [6]:
# забираємо годину з поля date
data['date'] = pd.to_datetime(data['date']).dt.normalize()

In [7]:
# раніше ми замінили 24 на 0 в даних, але через це в нас дата змінилася на 1 день, тепер це треба виправити
data.loc[data['hour'] == 0, 'date'] += pd.Timedelta(days=1)

In [8]:
# залишаємо тільки 4 дати, які будуть представляти свою пору року
data = data.loc[
    (data['date'] == '2020-01-14') | 
    (data['date'] == '2020-04-14') | 
    (data['date'] == '2020-07-14') |
    (data['date'] == '2020-10-14')]


In [9]:
# створюємо поле з назвою пори року
data['Сезон'] = ""
data.loc[data['date'] == '2020-01-14', 'Сезон'] = 'Зима'
data.loc[data['date'] == '2020-04-14', 'Сезон'] = 'Весна'
data.loc[data['date'] == '2020-07-14', 'Сезон'] = 'Літо'
data.loc[data['date'] == '2020-10-14', 'Сезон'] = 'Осінь'
data = data.drop(columns=['date'])

In [10]:
# для зручності зкомбінуємо дані з колонок з джерелами електроенергії в одну.
# є кілька варіантів як їх називати
sources = ["AES", "TEC", "VDE", "TES", "GES", "GAES_GEN"]
new_sources = ["АЕС", "ТЕЦ", "ВДЕ", "ТЕС", "ГЕС", "ГАЕС"]
new_sources_full = ["Атомна електростанції", "Теплоелектроцентралі", "Відновлювана енергетика", "Теплоелектростанції", "Гідроелектростанції", "Гідроакумулюючі електростанції"]
data_by_source = []

for i, source in enumerate(sources):
    temp_data = data[['hour', 'Сезон', source]].rename(columns={source: "production"})
    temp_data["Джерело енергії"] = new_sources[i]
    data_by_source.append(temp_data)
    
data = pd.concat(data_by_source, ignore_index=True)
print(data["Джерело енергії"].unique())

data.head()

['АЕС' 'ТЕЦ' 'ВДЕ' 'ТЕС' 'ГЕС' 'ГАЕС']


hour  Сезон  production Джерело енергії
0    23  Осінь      8553.0             АЕС
1    22  Осінь      8540.0             АЕС
2    21  Осінь      8561.0             АЕС
3    20  Осінь      8543.0             АЕС
4    19  Осінь      8530.0             АЕС

In [11]:
data['order'] = 0 
for i, source in enumerate(new_sources):
    current_subset = data['Джерело енергії'] == source
    data['order'] += (current_subset).astype('int32') * data[current_subset]['production'].mean()
    print(source, data[current_subset]['production'].mean())

АЕС 9035.010416666666
ТЕЦ 1428.8125
ВДЕ 838.71875
ТЕС 4088.1770833333335
ГЕС 655.59375
ГАЕС 172.71875


In [12]:
data.head()

hour  Сезон  production Джерело енергії        order
0    23  Осінь      8553.0             АЕС  9035.010417
1    22  Осінь      8540.0             АЕС  9035.010417
2    21  Осінь      8561.0             АЕС  9035.010417
3    20  Осінь      8543.0             АЕС  9035.010417
4    19  Осінь      8530.0             АЕС  9035.010417

In [16]:
alt.Chart(data, title={'text': "Генерація електроенегрії в Україні в залежності від години"}).mark_area().encode(
    x = alt.X('hour:N', axis=alt.Axis(title='Година', titleFontSize=14)),
    y = alt.Y('production', axis=alt.Axis(title='Вироблено енергії в МВт', titleFontSize=14)),
    color = alt.Color('Джерело енергії'),
    order = alt.Order('order:N', aggregate='sum', sort = 'descending')
).properties(width = 950/2, height = 500/2).facet(
    facet = alt.Facet('Сезон', sort=['Зима', 'Весна', 'Літо', 'Осінь']), columns = 2
#     column= alt.Column('Сезон', sort=['Зима', 'Весна', 'Літо', 'Осінь'])
).configure_title(fontSize=18)


alt.FacetChart(...)